In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/wu3d-dipression-detection/normal.json
/kaggle/input/wu3d-dipression-detection/depressed.json


In [4]:
# Open the JSON file
import json

with open("/kaggle/input/wu3d-dipression-detection/depressed.json", "r") as json_file:
    # Load the JSON data into a Python dictionary
    data_depressed_user = json.load(json_file)


with open("/kaggle/input/wu3d-dipression-detection/normal.json", "r") as json_file2:
    # Load the JSON data into a Python dictionary
    data_normal_user = json.load(json_file2)

KeyboardInterrupt: 

In [2]:
import csv
import json


with open('/kaggle/input/wu3d-dipression-detection/depressed.json') as json_file_dipressUser:
    jsondata = json.load(json_file_dipressUser)

data_file = open('depressUserConvertFile.csv', 'w', newline='')
csv_writer = csv.writer(data_file)

count = 0
for data in jsondata:
    if count == 0:
        header = data.keys()
        csv_writer.writerow(header)
        count += 1
    csv_writer.writerow(data.values())
data_file.close()

In [3]:
# Here we convert our json file into csv Normal User
with open('/kaggle/input/wu3d-dipression-detection/normal.json') as json_file_normal_User:
    jsondata = json.load(json_file_normal_User)

data_file = open('normalUserConvert.csv', 'w', newline='')
csv_writer = csv.writer(data_file)

count = 0
for data in jsondata:
    if count == 0:
        header = data.keys()
        csv_writer.writerow(header)
        count += 1
    csv_writer.writerow(data.values())
data_file.close()

In [ ]:
import ast
import pandas as pd

df = pd.read_csv('/kaggle/working/depressUserConvertFile.csv')

# df.head()

str_dict = df['tweets'][1]
dictionary = ast.literal_eval(str_dict)

In [ ]:
# Dipression - 1

tweet_list = []
for i in df['tweets']:
    dictionary = ast.literal_eval(str_dict)
    # print(dictionary)
    for j in dictionary:
        tweet_list.append({'texts':j['tweet_content'],'labels':1})

# Length of dipressed Tweets
len(tweet_list)


# Check & Display any dipressed tweet
# tweet_list[5]

In [ ]:
import ast
import pandas as pd

df_2 = pd.read_csv('/kaggle/working/normalUserConvert.csv')

# df.head()

str_dict = df_2['tweets'][0]
dictionary_2 = ast.literal_eval(str_dict)

In [4]:
import pandas as pd
from sklearn.utils import shuffle

# Read the CSV files into DataFrames
df_normal = pd.read_csv('/kaggle/working/normalUserConvert.csv')
df_deep = pd.read_csv('/kaggle/working/depressUserConvertFile.csv')

# Randomly select 3000 rows from each DataFrame
df_normal = df_normal.sample(n=3000, random_state=42)
df_deep = df_deep.sample(n=3000, random_state=42)

# Concatenate the sampled DataFrames
final_df = pd.concat([df_normal, df_deep])

# Shuffle the combined DataFrame
final_df = shuffle(final_df, random_state=42)

# Reset index after shuffling
final_df.reset_index(drop=True, inplace=True)

# Print the first few rows of the shuffled DataFrame
# print(df_combined_shuffled.head())
final_df.head(5)



,label,nickname,gender,profile,birthday,num_of_follower,num_of_following,all_tweet_count,original_tweet_count,repost_tweet_count,tweets
0,1,多吃一点小糖豆,女,给自己起个小名叫快快,巨蟹座,530,380,368,28,70,"[{'tweet_content': '年纪越大越想做一个小朋友', 'posting_ti..."
1,1,铁拳会说话,女,漂泊的灵魂,射手座,137,235,154,20,2,[{'tweet_content': '他喜欢看知音身为 一个堂堂七尺男儿却喜欢看知音一定是...
2,0,梦想家脸庞eyti,男,在任何科学上的雏形，都有它双重的形象：胚胎时的丑恶，萌芽时的美丽。,无,75,117,87,75,10,"[{'tweet_content': '美丽是一步步来的上海隆鼻哪家整形医院好', 'pos..."
3,0,小MM-Qun,女,Group,01-01,190,410,1834,99,1,"[{'tweet_content': '母親節 準備好了麼～', 'posting_time..."
4,1,Outofcuriosity_,女,This is a blog only written original contents....,无,1,9,6,12,0,[{'tweet_content': '惊了 我有点被这抗抑郁的药吓到了中午吃了这心情也太好...


In [5]:
final_df = final_df[['label','tweets']]

print(len(final_df))
final_df.head()

6000


,label,tweets
0,1,"[{'tweet_content': '年纪越大越想做一个小朋友', 'posting_ti..."
1,1,[{'tweet_content': '他喜欢看知音身为 一个堂堂七尺男儿却喜欢看知音一定是...
2,0,"[{'tweet_content': '美丽是一步步来的上海隆鼻哪家整形医院好', 'pos..."
3,0,"[{'tweet_content': '母親節 準備好了麼～', 'posting_time..."
4,1,[{'tweet_content': '惊了 我有点被这抗抑郁的药吓到了中午吃了这心情也太好...


In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import XLNetTokenizer, XLNetForSequenceClassification, AdamW
import torch
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm

# Tokenize the sentences
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')

# Find the maximum length of the sequences in the dataset
max_length = max(len(tokenizer.encode(text)) for text in final_df['tweets'])
print("Maximum sequence length:", max_length)

# Set a maximum length for truncation
max_length = min(max_length, 128)  # Limit the maximum length to 128 to conserve memory

encoded_data = tokenizer(final_df['tweets'].tolist(), padding=True, truncation=True, max_length=max_length, return_tensors='pt')

# Split the data into train, validation, and test sets
train_texts, val_texts, train_labels, val_labels = train_test_split(encoded_data['input_ids'], final_df['label'], test_size=0.2, random_state=42)
val_texts, test_texts, val_labels, test_labels = train_test_split(val_texts, val_labels, test_size=0.5, random_state=42)

# Convert labels to NumPy arrays
train_labels = train_labels.to_numpy()
val_labels = val_labels.to_numpy()
test_labels = test_labels.to_numpy()

# Define the data loaders
train_dataset = TensorDataset(train_texts, torch.tensor(train_labels))  # Ensure train_labels is converted to tensor
val_dataset = TensorDataset(val_texts, torch.tensor(val_labels))  # Ensure val_labels is converted to tensor
test_dataset = TensorDataset(test_texts, torch.tensor(test_labels))  # Ensure test_labels is converted to tensor

# Define the model architecture
model = XLNetForSequenceClassification.from_pretrained('xlnet-base-cased', num_labels=2)

# Define training parameters
optimizer = AdamW(model.parameters(), lr=2e-5)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Define data loaders with smaller batch sizes
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)  # Reduced batch size to 8
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)  # Reduced batch size to 8
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)  # Reduced batch size to 8

# Training loop
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    train_accuracy = 0
    train_total = 0
    train_loss = 0
    progress_bar = tqdm(train_loader, desc=f'Epoch {epoch + 1}/{num_epochs}', leave=False)
    for batch in progress_bar:
        inputs, labels = batch
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs, labels=labels)
        loss = outputs.loss
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.logits, 1)
        train_accuracy += (predicted == labels).sum().item()
        train_total += labels.size(0)
        train_acc = train_accuracy / train_total
        progress_bar.set_postfix(train_acc=train_acc, train_loss=train_loss / train_total)
    train_accuracy /= train_total

    # Validation loop
    model.eval()
    val_accuracy = 0
    val_total = 0
    val_loss = 0
    with torch.no_grad():
        for batch in val_loader:
            inputs, labels = batch
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            logits = outputs.logits
            _, predicted = torch.max(logits, 1)
            val_accuracy += (predicted == labels).sum().item()
            val_total += labels.size(0)
            val_loss += torch.nn.functional.cross_entropy(logits, labels).item()
    val_accuracy /= val_total
    print(f'Epoch {epoch + 1}/{num_epochs}, Train Accuracy: {train_accuracy:.4f}, Train Loss: {train_loss / train_total:.4f}, Validation Accuracy: {val_accuracy:.4f}, Validation Loss: {val_loss / val_total:.4f}')

# Testing loop
model.eval()
test_accuracy = 0
test_total = 0
test_loss = 0
with torch.no_grad():
    for batch in test_loader:
        inputs, labels = batch
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        logits = outputs.logits
        _, predicted = torch.max(logits, 1)
        test_accuracy += (predicted == labels).sum().item()
        test_total += labels.size(0)
        test_loss += torch.nn.functional.cross_entropy(logits, labels).item()
test_accuracy /= test_total
print(f'Test Accuracy: {test_accuracy:.4f}, Test Loss: {test_loss / test_total:.4f}')


Maximum sequence length: 38518


pytorch_model.bin:   0%|          | 0.00/467M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'logits_proj.weight', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_dep

Epoch 1/50, Train Accuracy: 0.7554, Train Loss: 0.0707, Validation Accuracy: 0.7883, Validation Loss: 0.0652


Epoch 2/50, Train Accuracy: 0.7562, Train Loss: 0.0697, Validation Accuracy: 0.7883, Validation Loss: 0.0627


...

Epoch 46/50, Train Accuracy: 0.9640, Train Loss: 0.0121, Validation Accuracy: 0.7633, Validation Loss: 0.1344


Epoch 47/50, Train Accuracy: 0.9675, Train Loss: 0.0121, Validation Accuracy: 0.7267, Validation Loss: 0.1348


Epoch 48/50, Train Accuracy: 0.9671, Train Loss: 0.0114, Validation Accuracy: 0.7783, Validation Loss: 0.1422


Epoch 49/50, Train Accuracy: 0.9744, Train Loss: 0.0092, Validation Accuracy: 0.7600, Validation Loss: 0.1369


Epoch 50/50, Train Accuracy: 0.9775, Train Loss: 0.0084, Validation Accuracy: 0.7600, Validation Loss: 0.1425
Test Accuracy: 0.7650, Test Loss: 0.1327


In [8]:
from sklearn.metrics import classification_report, confusion_matrix

# Evaluate the model
y_pred = model.predict(X_test)
y_pred_binary = (y_pred > 0.5).astype("int32")

# Calculate evaluation metrics
print("Classification Report:")
print(classification_report(y_test, y_pred_binary))

print("Confusion Matrix:")
conf_matrix = confusion_matrix(y_test, y_pred_binary)
print(conf_matrix)

# Calculate precision, recall, and F1-score
tn, fp, fn, tp = conf_matrix.ravel()
precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1_score = 2 * (precision * recall) / (precision + recall)

print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1_score)


AttributeError: 'XLNetForSequenceClassification' object has no attribute 'predict'

----------------------------
## XLEXNET 2.0

In [ ]:
import ast
import pandas as pd

df = pd.read_csv('/content/depressUserConvertFile.csv')

str_dict = df['tweets'][1]
dictionary = ast.literal_eval(str_dict)

# dictionary
print(dictionary)

tweet_list_1 = []
for i in df['tweets']:
    dictionary = ast.literal_eval(str_dict)
    # print(dictionary)
    for j in dictionary:
        # tweet_list_1.append({'posting time':j['posting_time'],'labels':1})
        tweet_list_1.append({'posting time':j['posting_time'],'orignal tweet':j['tweet_is_original'], 'labels':1})

# Check & Display any dipressed tweet
print(len(tweet_list_1))

In [ ]:
import ast
import pandas as pd

df_2 = pd.read_csv('/content/normalUserConvert.csv')

str_dict = df_2['tweets'][0]
dictionary_2 = ast.literal_eval(str_dict)

tweet_list_normal = []
for i in df['tweets']:
    dictionary_2 = ast.literal_eval(str_dict)
    # print(dictionary)
    for j in dictionary_2:
        tweet_list_normal.append({'Post Timing':j['posting_time'], 'orignal tweet':j['tweet_is_original'], 'labels':0})

# Check & Display any dipressed tweet
print(len(tweet_list_normal))

In [ ]:
import pandas as pd
from sklearn.utils import shuffle

# Read the tweet_list_normal and tweet_list_dipressed into DataFrames
df_normal_posting = pd.DataFrame(tweet_list_normal)
df_deep_posting = pd.DataFrame(tweet_list_1)

# Randomly select 3000 rows from each DataFrame
df_normal_sampled = df_normal_posting.sample(n=3000, random_state=42)
df_deep_sampled = df_deep_posting.sample(n=3000, random_state=42)

# Concatenate the sampled DataFrames
df_combined = pd.concat([df_normal_sampled, df_deep_sampled])

# Shuffle the combined DataFrame
df_combined_shuffled = shuffle(df_combined, random_state=42)

# Reset index after shuffling
df_combined_shuffled.reset_index(drop=True, inplace=True)

# Print the first few rows of the shuffled DataFrame
# print(df_combined_shuffled.head())
df_combined_shuffled.head(10)

final_df = df_combined_shuffled[['labels','Post Timing', 'orignal tweet']]

print(len(final_df))
final_df.head()

In [ ]:
import pandas as pd
import numpy as np

# Convert 'Post Timing' column to datetime format, handle parsing errors
final_df['Post Timing'] = pd.to_datetime(final_df['Post Timing'], errors='coerce')

# Remove rows with NaT values (parsing errors)
final_df = final_df.dropna(subset=['Post Timing'])

# Calculate features for each label separately
social_features = {}

# Iterate over unique labels
for label in final_df['labels'].unique():
    # Subset the DataFrame for the current label
    subset_df = final_df[final_df['labels'] == label]

    # Calculate features
    # Proportion of original tweets
    proportion_original = subset_df['orignal tweet'].value_counts(normalize=True)['True']

    # Late-night posting (posts made between 11 PM and 6 AM)
    late_night_posts = subset_df[(subset_df['Post Timing'].dt.hour >= 23) | (subset_df['Post Timing'].dt.hour <= 6)]
    proportion_late_night_posts = len(late_night_posts) / len(subset_df)

    # Posting frequency (per week)
    posting_frequency_per_week = len(subset_df) / ((subset_df['Post Timing'].max() - subset_df['Post Timing'].min()).days / 7)

    # Standard deviation of posting time
    std_dev_posting_time = subset_df['Post Timing'].diff().dt.total_seconds().std()

    # Store features for the current label
    social_features[label] = {
        'Proportion of Original Tweets': proportion_original,
        'Proportion of Late-night Posts': proportion_late_night_posts,
        'Posting Frequency (per week)': posting_frequency_per_week,
        'Standard Deviation of Posting Time': std_dev_posting_time
    }

# Convert features dictionary to DataFrame
social_features_df = pd.DataFrame.from_dict(social_features, orient='index')

# Display the social features
print("Social Behavior-based Features:")
print(social_features_df)

In [ ]:
from sklearn.model_selection import train_test_split
from transformers import XLNetTokenizer, XLNetForSequenceClassification, AdamW
import torch
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm

# Load the dataset
# final_df = pd.read_csv('your_dataset.csv')

# Combine 'Post Timing' and 'orignal tweet' features
final_df['features'] = final_df['Post Timing'].astype(str) + ' ' + final_df['orignal tweet']

# Tokenize the sentences
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')

# Find the maximum length of the sequences in the dataset
max_length = max(len(tokenizer.encode(text)) for text in final_df['features'])
print("Maximum sequence length:", max_length)

# Set a maximum length for truncation
max_length = min(max_length, 64)  # Limit the maximum length to 128 to conserve memory

encoded_data = tokenizer(final_df['features'].tolist(), padding=True, truncation=True, max_length=max_length, return_tensors='pt')

# Split the data into train, validation, and test sets
train_texts, val_texts, train_labels, val_labels = train_test_split(encoded_data['input_ids'], final_df['labels'], test_size=0.2, random_state=42)
val_texts, test_texts, val_labels, test_labels = train_test_split(val_texts, val_labels, test_size=0.5, random_state=42)

# Convert labels to NumPy arrays
train_labels = train_labels.to_numpy()
val_labels = val_labels.to_numpy()
test_labels = test_labels.to_numpy()

# Define the data loaders
train_dataset = TensorDataset(train_texts, torch.tensor(train_labels))  # Ensure train_labels is converted to tensor
val_dataset = TensorDataset(val_texts, torch.tensor(val_labels))  # Ensure val_labels is converted to tensor
test_dataset = TensorDataset(test_texts, torch.tensor(test_labels))  # Ensure test_labels is converted to tensor

# Define the model architecture
model = XLNetForSequenceClassification.from_pretrained('xlnet-base-cased', num_labels=2)

# Define training parameters
optimizer = AdamW(model.parameters(), lr=2e-5)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Define data loaders with smaller batch sizes
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)  # Reduced batch size to 8
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False)  # Reduced batch size to 8
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)  # Reduced batch size to 8

# Training loop
num_epochs = 2
for epoch in range(num_epochs):
    model.train()
    train_accuracy = 0
    train_total = 0
    train_loss = 0
    progress_bar = tqdm(train_loader, desc=f'Epoch {epoch + 1}/{num_epochs}', leave=False)
    for batch in progress_bar:
        inputs, labels = batch
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs, labels=labels)
        loss = outputs.loss
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.logits, 1)
        train_accuracy += (predicted == labels).sum().item()
        train_total += labels.size(0)
        train_acc = train_accuracy / train_total
        progress_bar.set_postfix(train_acc=train_acc, train_loss=train_loss / train_total)
    train_accuracy /= train_total

    # Validation loop
    model.eval()
    val_accuracy = 0
    val_total = 0
    val_loss = 0
    with torch.no_grad():
        for batch in val_loader:
            inputs, labels = batch
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            logits = outputs.logits
            _, predicted = torch.max(logits, 1)
            val_accuracy += (predicted == labels).sum().item()
            val_total += labels.size(0)
            val_loss += torch.nn.functional.cross_entropy(logits, labels).item()
    val_accuracy /= val_total
    print(f'Epoch {epoch + 1}/{num_epochs}, Train Accuracy: {train_accuracy:.4f}, Train Loss: {train_loss / train_total:.4f}, Validation Accuracy: {val_accuracy:.4f}, Validation Loss: {val_loss / val_total:.4f}')

# Testing loop
model.eval()
test_accuracy = 0
test_total = 0
test_loss = 0
with torch.no_grad():
    for batch in test_loader:
        inputs, labels = batch
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        logits = outputs.logits
        _, predicted = torch.max(logits, 1)
        test_accuracy += (predicted == labels).sum().item()
        test_total += labels.size(0)
        test_loss += torch.nn.functional.cross_entropy(logits, labels).item()
test_accuracy /= test_total
print(f'Test Accuracy: {test_accuracy:.4f}, Test Loss: {test_loss / test_total:.4f}')


In [ ]:
# Plot loss curves
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

# Evaluate the model
y_pred = model.predict(X_test)
y_pred_binary = (y_pred > 0.5).astype("int32")

# Calculate evaluation metrics
print("Classification Report:")
print(classification_report(y_test, y_pred_binary))

print("Confusion Matrix:")
conf_matrix = confusion_matrix(y_test, y_pred_binary)
print(conf_matrix)

# Calculate precision, recall, and F1-score
tn, fp, fn, tp = conf_matrix.ravel()
precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1_score = 2 * (precision * recall) / (precision + recall)

print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1_score)
